## 关于使用逻辑回归完成猫图识别作业笔记


### 关于h5文件

#### h5文件简介
H5文件其实就是HDF5，是用于存储和分发科学数据的一种多对象文件格式，在做数据分析之前，必须知道自己的数据形式，才能选择适当的方式来处理数据，使用这种文件格式的优点有：
1. 跨平台性，HDF是一个与平台无关的文件格式，不需要任何转换就能在不同平台上使用。
2. 自述性：对于一个HDF文件里的每一个数据对象，有关于该数据的综合信息，允许应用程序解释HDF文件的结构和内容。
3. 通用性：许多数据类型都可以被潜在一个hdf文件里，使用合适的数据结构，符号，数字和图形数据都能同时存在一个HDF文件里。
4. 扩展性：特别容易容纳将来新增的数据模式，容易与其他标准格式兼容。

#### HDF5文件组织：
一个HDF5文件就是由两种基本数据对象组成的：组(group)和数据集(datasets),一个组包含0个或多个HDF5对象并且支持元数据，数据集指的是数据元素的一维数组并且也支持元数据。

关于HDF5说明的一个网页：http://blog.csdn.net/mrhiuser/article/details/69603826
其中关于如何制作HDF5文件，编译代码文件时注意g++命令可以在linux环境下安装软件，同时还要安装g++命令，最后可以得到HDF5文件。

#### 如何打开后缀为h5的文件
如何打开.h5文件，可以使用一个软件，HDFView用于浏览和编辑HDF4和HDF5文件。安装的时候可能会提示，找不到某个路径下的文件，可以发现那个路径下确实没有，找到安装到的内容复制到提示的地方，软件则可以使用了。

### 识别猫图作业
数据集：
训练数据集：train_cat.h5
测试数据集：test_cat.h5
使用普通打开文本的方式打开发现内部都是一些数字，并不能看懂，使用下好的HDFView打开，则可以看到里面的内容。点开文件发现，测试文件和训练文件都有一样的结构，每个文件中有两个二维表和一个文本。可以发现，text里面说明了0代表非猫，1代表猫，test_set_x代表特征数据，一行代表一个样本的特征数据，有多少行则有多少个样本。test_set_y则代表对应样本的标签，即监督学习里的训练测试数据原本的真实分类类标。

![HDFView打开.h5文件](http://upload-images.jianshu.io/upload_images/1779926-4a212fd643b0d2e8.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

![数据文件内容](http://upload-images.jianshu.io/upload_images/1779926-84bcf96eb641656b.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)


核心代码：
```python
# 相关包引用
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset  
%matplotlib inline

# 加载数据
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

# 查看数据集中的一个例子
index = 25
print (np.squeeze(train_set_y[:, index]))
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:, index]) + ", it's a '" + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  "' picture.")

# 查看数据集的结构，以及数据的多少
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]
print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))


# 整形数据，使数据扁平化
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0],-1).T


# 标准化数据
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

# 函数1： sigmoid()函数
def sigmoid(z):
    s = 1.0 / (1.0 + np.exp(-z ))
    return s

# 函数2： 初始化参数函数，w，b
def initialize_with_zeros(dim):
    w = np.zeros((dim,1))   
    b = 0
    return w, b

# 函数3：传播函数，包括正向和反向传播，计算cost和梯度
def propagate(w, b, X, Y):
    m = X.shape[1]
    A = sigmoid(np.dot(w.T,X)+b )                                
    cost = -np.sum(np.dot(np.log(A),Y.T)+np.dot(np.log(1.0-A),(1.0-Y).T))/m                   # compute cost

    dw = 1.0/m* np.dot(X,(A-Y).T)
    db = 1.0/m* np.sum(A-Y)
    cost = np.squeeze(cost)   
    grads = {"dw": dw,
             "db": db}
    return grads, cost

# 函数4：优化函数：通过迭代学习参数并且优化
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):  
    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
    
        w = w - learning_rate*dw
        b = b- learning_rate*db
        
        if i % 100 == 0:
            costs.append(cost)
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    grads = {"dw": dw,
             "db": db}
    return params, grads, costs

# 函数5：预测函数：predict(w,b，X)通过优化后的w，b进行预测
def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T, X)+b)

    for i in range(A.shape[1]):
        if A[0,i]>=0.5:
            Y_prediction[0,i]=1
        else:
             Y_prediction[0,i]=0
        pass
    return Y_prediction

# 函数6：总模型，调用前面写好的函数，对训练集和测试集分别计算正确率
def model(X_train, Y_train, X_test, Y_test, num_iterations, learning_rate, print_cost):
    w, b = initialize_with_zeros(X_train.shape[0])
    parameters, grads, costs = optimize(w,b,X_train, Y_train,num_iterations, learning_rate, print_cost)
    w = parameters["w"]
    b = parameters["b"]

    Y_prediction_test = predict(w,b,X_test)
    Y_prediction_train = predict(w,b,X_train)
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}  
    return d


# 直接调用模型总函数，开始运行代码得到结果
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)
```



![逻辑回归识别猫图代码测试结果](http://upload-images.jianshu.io/upload_images/1779926-cfe7950cba86b70c.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)


## 总结：
1. 对数据集进行预处理是很重要的。
2. 你实现的每个功能分别是：initialize()，propagate()，optimize()，predict()。然后你建立一个model()。
3. 调整学习率（这是一个例子，一个“超参数”）可以对算法有很大的不同。


最后，你可以继续进行相关尝试，对模型进行优化：
- 学习率和迭代次数
- 尝试不同的初始化方法并比较结果
- 测试其他预处理（中心数据，或者将每一行的标准偏差）